In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as datetime

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import StepLR
import torch.optim as optim

from sklearn.preprocessing import MinMaxScaler

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
ds = xr.open_dataset('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/ampere_omni_2010_2013.nc')
ds

In [ ]:
jpar = ds['jPar'].values.flatten()
jpar = jpar[::1000]
sns.histplot(jpar, bins=100, kde=False)

In [ ]:
# Initialize scalers for the target variable and input variables
input_scaler = MinMaxScaler()

# Extract the target variable and reshape for scaling
target_var = ds['jPar'].values  # shape (t (n), mlat (50), mlt (24))

# Extract and scale input variables (variables that are dependent only on 'dt')
input_vars = ['BX_GSE', 'BY_GSE', 'BZ_GSE', 'flow_speed', 'proton_density', 'AL_INDEX', 'AU_INDEX', 'SYM_H', 'ASY_H', 'F10.7', 'Kp']
input_data = np.array([ds[var].values for var in input_vars]).T  # shape (22320, number_of_vars)
input_data_scaled = input_scaler.fit_transform(input_data)

def create_sequences(target_data, input_data, lookback=30):
    X, y = [], []
    for i in range(len(target_data) - lookback):
        X.append(input_data[i:i+lookback].T)
        y.append(target_data[i+lookback])

    return np.array(X), np.array(y)

lookback = 30
X, y = create_sequences(target_var, input_data_scaled, lookback=lookback)

X.shape, y.shape

In [ ]:
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# Calculate split indices
train_idx = int(len(X) * train_ratio)
val_idx = int(len(X) * (train_ratio + val_ratio))

# Perform the split
X_train, X_val, X_test = X[:train_idx], X[train_idx:val_idx], X[val_idx:]
y_train, y_val, y_test = y[:train_idx], y[train_idx:val_idx], y[val_idx:]

X_train.shape, X_val.shape, X_test.shape

In [ ]:
def random_time_series_split(X, y, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1, segment_length=720):
    total_segments = len(X) // segment_length
    total_samples = total_segments * segment_length
    
    # Shuffle indices to randomly select segments
    indices = np.arange(total_segments)
    np.random.shuffle(indices)
    
    # Calculate sizes of train, validation, and test sets
    train_size = int(total_segments * train_ratio)
    val_size = int(total_segments * val_ratio)
    test_size = total_segments - train_size - val_size
    
    # Select indices for train, validation, and test sets
    train_indices = indices[:train_size]
    val_indices = indices[train_size:train_size + val_size]
    test_indices = indices[train_size + val_size:]
    
    # Function to extract segments
    def extract_segments(indices):
        X_segments = []
        y_segments = []
        for idx in indices:
            start_idx = idx * segment_length
            X_segment = X[start_idx:start_idx + segment_length]
            y_segment = y[start_idx:start_idx + segment_length]
            X_segments.append(X_segment)
            y_segments.append(y_segment)
        return np.concatenate(X_segments), np.concatenate(y_segments)
    
    # Extract train, validation, and test sets
    X_train, y_train = extract_segments(train_indices)
    X_val, y_val = extract_segments(val_indices)
    X_test, y_test = extract_segments(test_indices)
    
    return X_train, y_train, X_val, y_val, X_test, y_test

X_train, y_train, X_val, y_val, X_test, y_test = random_time_series_split(X, y)
X_train.shape, X_val.shape, X_test.shape

In [ ]:

# Convert data to PyTorch tensors and move to device
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
X_val = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val = torch.tensor(y_val, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

# Create DataLoader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=False)

val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

In [ ]:
'''
class LSTMModel(nn.Module):
    def __init__(self):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size=lookback, hidden_size=64, num_layers=3, batch_first=True)
        self.fc = nn.Linear(64, 50*24)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])
        x = x.view(-1, 50, 24)
        return x'''

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_stacked_layers):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_stacked_layers = num_stacked_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_stacked_layers, 
                            batch_first=True)
        
        self.fc = nn.Linear(hidden_size, 40*24)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
        c0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        out = out.view(-1, 40, 24)
        return out
    
    def reset_states(self):
        # Reset the internal states of the LSTM layer
        self.lstm.reset_parameters()

# Instantiate the model with the correct input size
model = LSTM(lookback, 64, 2)
model.to(device)
model

In [ ]:
loss_function = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = StepLR(optimizer, step_size=10, gamma=0.6)

In [ ]:
# Train the model
num_epochs = 100
train_losses = []
val_losses = []

#early stopping
best_val_loss = float('inf')
patience = 5
counter = 0

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0

    for X_batch, y_batch in train_loader:
        # Forward pass
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        train_outputs = model(X_batch)
        loss = loss_function(train_outputs, y_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    # Step the scheduler
    scheduler.step()

    train_loss = epoch_loss / len(train_loader)
    train_losses.append(train_loss)

    # Validation
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            val_outputs = model(X_batch)
            val_loss += loss_function(val_outputs, y_batch).item()

    val_loss /= len(val_loader)
    val_losses.append(val_loss)

    #Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
    else:
        counter += 1
        print(f'Counter is at {counter}')
        
        if counter == patience:
            print(f'Validation loss did not improve for {patience} epochs. Stopping training.')
            break
        else:
            continue

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

In [ ]:
#test
model.eval()

test_loss = 0.0
predictions = []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        #batch_X = batch_X.permute(0, 2, 1)

        output = model(batch_X)
        loss = loss_function(output, batch_y)
        test_loss += loss.item()
        predictions.append(output)

predictions = torch.cat(predictions)
predictions = predictions.cpu().numpy()

avg_test_loss = test_loss / len(test_loader)
microamp = r'$\mu$A'
print(f'Test Loss: {np.sqrt(avg_test_loss):.3f} {microamp}')

mlat_idx = 39  #0 through 49
mlt_idx = 1 #0 through 23

#y_test = y_test.cpu().numpy()

predictions_mean = np.mean(predictions, axis=0)
y_test_mean = np.mean(y_test, axis=0)

R = np.corrcoef(y_test_mean[:, mlat_idx, mlt_idx], predictions_mean[:, mlat_idx, mlt_idx])[0,1]

R = np.corrcoef(y_test[:, mlat_idx, mlt_idx], predictions[:, mlat_idx, mlt_idx])[0,1]
MAE = np.mean(np.abs(y_test[:, mlat_idx, mlt_idx] - predictions[:, mlat_idx, mlt_idx]))
RMSE = np.sqrt(np.mean((y_test[:, mlat_idx, mlt_idx] - predictions[:, mlat_idx, mlt_idx])**2))
NRMSE = RMSE / (np.max(y_test[:, mlat_idx, mlt_idx]) - np.min(y_test[:, mlat_idx, mlt_idx]))
print(f'R: {R:.3f}, RMSE: {RMSE:.3f}, NRMSE: {NRMSE:.3f}, MAE: {MAE:.3f}')


plt.figure(figsize=(10, 4))
plt.plot(y_test[:, mlat_idx, mlt_idx], label='Actual')
plt.plot(predictions[:, mlat_idx, mlt_idx], label='Predicted')
plt.ylim(-1,1)
plt.legend()

In [ ]:
predictions_mean = np.mean(predictions, axis=0)
y_test_mean = np.mean(y_test, axis=0)
predictions_mean.shape, y_test_mean.shape

In [ ]:
fac = predictions_mean

def dt_to_str(dt):
    dt_1 = dt[0]
    dt_2 = dt[1] 
    time_1 = pd.to_datetime(dt_1)
    time_2 = pd.to_datetime(dt_2)
    str_time_1 = time_1.strftime('%Y-%m-%d %H:%M')
    str_time_2 = time_2.strftime('%Y-%m-%d %H:%M')

    return str_time_1, str_time_2

#dt = open_amp['dt'].values
#start_time, end_time = dt_to_str(dt)

fac = fac.reshape(y_test_mean.shape[1], y_test_mean.shape[0]).T # reshape and transpose
fac = np.flipud(fac) # flip the array upside down
theta = np.linspace(0, 360, y_test_mean.shape[1]) - 90 # rotate by 90 degrees
theta = np.radians(theta) # convert to radians
r = 90 - np.linspace(50, 90, y_test_mean.shape[0]) #convert to colat

#cmap = mcolors.LinearSegmentedColormap.from_list("my_colormap", ["blue","blue", "white", "white","red","red"])
cmap = 'bwr'

fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(6, 5))

c = ax.contourf(theta, r, fac, cmap=cmap, levels=np.linspace(-0.5,.5 , 100))
ax.set_ylim([0,40])
ax.set_yticks([0, 10, 20, 30,40])
ax.set_yticklabels(["90°", "80°", "70°", "60°","50° MLAT"])
ax.set_xlim([-np.pi, np.pi])
ax.set_xticks(np.linspace(-np.pi, np.pi, 9)[1:])
ax.set_xticklabels(["21", "0 MLT \nMidnight", "3", "6 \n  Dawn", "9", "12 MLT \nMidday", "15", "18 \nDusk"])
ax.grid(True, linestyle='-.', alpha=0.7)
#ax.set_title(f"{start_time} - {end_time}", pad=10, fontsize=11.5)


plt.colorbar(c, ax=ax, label='J$_\parallel$ (FAC) [µA/m$^2$]', shrink=0.3, pad = 0.12, 
             ticks=[-0.5, 0,  0.5], 
             orientation='horizontal')

plt.tight_layout()

In [ ]:
plt.contourf(fac, cmap=cmap, levels=np.linspace(-0.5, 0.5, 100))
